# 확률및 통계1 - 대체인정 실습

### 실습: statsmodels에 의한 회귀분석

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.formula.api as smf

%precision 3
%matplotlib inline

In [3]:
df = pd.read_csv('data/ch12_scores_reg.csv')
n = len(df)
print(n)
df.head()

20


,quiz,final_test,sleep_time,school_method
0,4.2,67,7.2,bus
1,7.2,71,7.9,bicycle
2,0.0,19,5.3,bus
3,3.0,35,6.8,walk
4,1.5,35,7.5,walk


##### 회귀분석에서는 “기말고사에 영향을 끼치는 주요 요인은 쪽지 시험의 평균 점수”라고 현상을 단순화

> * 설명(독립)변수:  쪽지 시험(quiz)
> * 반응(종속)변수:  기말고사(final_test)

In [4]:
formula = 'final_test ~ quiz'
result = smf.ols(formula, df).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             final_test   R-squared:                       0.676
Model:                            OLS   Adj. R-squared:                  0.658
Method:                 Least Squares   F-statistic:                     37.61
Date:                Thu, 10 Aug 2023   Prob (F-statistic):           8.59e-06
Time:                        12:17:53   Log-Likelihood:                -76.325
No. Observations:                  20   AIC:                             156.7
Df Residuals:                      18   BIC:                             158.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     23.6995      4.714      5.028      0.000      13.796      33.603
quiz           6.5537      1.069      6.133      0.000       4.309       8.799
==============================================================================
Omnibus:                        2.139   Durbin-Watson:                   1.478
Prob(Omnibus):                  0.343   Jarque-Bera (JB):                1.773
Skew:                           0.670   Prob(JB):                        0.412
Kurtosis:                       2.422   Cond. No.                         8.32
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### 회귀계수의 추정

In [5]:
x = np.array(df['quiz'])
y = np.array(df['final_test'])
p = 1

In [6]:
X = np.array([np.ones_like(x), x]).T
X

array([[1. , 4.2],
       [1. , 7.2],
       [1. , 0. ],
       [1. , 3. ],
       [1. , 1.5],
       [1. , 0.9],
       [1. , 1.9],
       [1. , 3.5],
       [1. , 4. ],
       [1. , 5.4],
       [1. , 4.2],
       [1. , 6.9],
       [1. , 2. ],
       [1. , 8.8],
       [1. , 0.3],
       [1. , 6.7],
       [1. , 4.2],
       [1. , 5.6],
       [1. , 1.4],
       [1. , 2. ]])

In [7]:
beta0_hat, beta1_hat = np.linalg.lstsq(X, y)[0]
beta0_hat, beta1_hat

C:\Users\신재현\AppData\Local\Temp\ipykernel_6584\1547428137.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  beta0_hat, beta1_hat = np.linalg.lstsq(X, y)[0]


(23.699, 6.554)

In [8]:
y_hat = beta0_hat + beta1_hat * x
eps_hat = y -y_hat

In [9]:
s_var = np.var(eps_hat, ddof = p+1)
s_var

134.290

### 회귀계수의 추정

* 점추정

In [10]:
C0, C1 = np.diag(np.linalg.pinv(np.dot(X.T, X)))

In [11]:
np.sqrt(s_var * C0), np.sqrt(s_var * C1)

(4.714, 1.069)

In [12]:
rv = stats.t(n-2)

lcl = beta0_hat - rv.isf(0.025) * np.sqrt(s_var * C0)
hcl = beta0_hat - rv.isf(0.975) * np.sqrt(s_var * C0)
lcl, hcl

(13.796, 33.603)

In [13]:
rv = stats.t(n-2)

lcl = beta1_hat - rv.isf(0.025) * np.sqrt(s_var * C1)
hcl = beta1_hat - rv.isf(0.975) * np.sqrt(s_var * C1)
lcl, hcl

(4.309, 8.799)

In [14]:
t = beta1_hat / np.sqrt(s_var * C1)
t

6.133

In [15]:
1 - rv.cdf(t) * 2

-1.000

### 결정계수(R^2)

In [16]:
y_hat = np.array(result.fittedvalues)
y_hat

array([51.225, 70.886, 23.699, 43.361, 33.53 , 29.598, 36.152, 46.638,
       49.914, 59.09 , 51.225, 68.92 , 36.807, 81.372, 25.666, 67.61 ,
       51.225, 60.4  , 32.875, 36.807])

In [17]:
eps_hat = np.array(result.resid)
eps_hat

array([ 15.775,   0.114,  -4.699,  -8.361,   1.47 ,  10.402, -13.152,
        -9.638, -10.914,  -4.09 , -11.225,   1.08 ,  -7.807,   6.628,
        21.334,   9.39 ,   0.775,  -5.4  , -14.875,  23.193])

In [19]:
# 잔차 제곱합
np.sum(eps_hat ** 2)

2417.228

In [20]:
# 총변동
total_var = np.sum((y - np.mean(y))**2)
# 회귀변동
exp_var = np.sum((y_hat - np.mean(y))**2)
# 잔차변동
unexp_var = np.sum(eps_hat ** 2)

In [21]:
# 결정계수
exp_var / total_var

0.676

In [22]:
np.corrcoef(x, y)[0, 1] ** 2

0.676

### F 검정

In [23]:
# F 검정통계량
f = (exp_var / p) / (unexp_var / (n - p - 1))
f

37.615

In [24]:
# p 값
rv = stats.f(p, n-p-1)
1 - rv.cdf(f)

0.000

In [25]:
# 왜도
stats.skew(eps_hat)

0.670

In [26]:
# 첨도
stats.kurtosis(eps_hat, fisher=False)

2.422